# Experiment 12.2: Poisson Ablations

In [ ]:
import jax
import jax.numpy as jnp

jax.config.update("jax_default_matmul_precision", "highest")

from src.utils import _get_adam, _get_pde_collocs, _get_ic_collocs, model_eval, count_params, _get_colloc_indices, grad_norm
from src.wrappers import PoissonModel

import numpy as np
from jax import device_get

import optax
from flax import nnx

import pickle
import time

import os

# Create the directory if it doesn't exist
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

result_file = os.path.join(results_dir, "ablations_poisson.pkl")

plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

RESULTS = dict()
        
seed = 42

In [ ]:
def summarize_results(results, rbn):

    l2_values = []
    for run in rbn.keys():
        l2_values.append(rbn[run]['l2'])

    mean = np.mean(l2_values)
    sem = np.std(l2_values, ddof=1) / np.sqrt(len(l2_values))

    print(f"Original:\t L^2 = {mean:.3e}  Error = {sem:.3e}")
    
    for ablation, runs in results.items():
        l2_values = [v['l2'] for v in runs.values()]
        mean = np.mean(l2_values)
        sem = np.std(l2_values, ddof=1) / np.sqrt(len(l2_values))
        print(f"{ablation}:\t L^2 = {mean:.3e}  Error = {sem:.3e}")

## Parameters

In [ ]:
# Training epochs
num_epochs = 100_000

# Scheduler configurations
learning_rate = 1e-3
decay_steps = 2000
decay_rate = 0.9
warmup_steps = 1000

# Define resampling parameters
batch_size = 2**12
f_resample = 2000
rad_a = 1.0
rad_c = 1.0

# Define RBA parameters
RBA_gamma = 0.999
RBA_eta = 0.01

# Define model parameters
n_in = 2
n_out = 1
D = 5
period_axes = None
sine_D = 5
init_scheme = {'type': 'glorot', 'gain': None, 'norm_pow': 0, 'distribution': 'uniform', 'sample_size': 10000}
n_hidden = 16
num_blocks = 6

## $\omega = 1$

In [ ]:
from src.equations import poisson_1_res as pde_res

In [ ]:
# Collocation points and ICs
collocs_pool = _get_pde_collocs(ranges = [(-1,1), (-1,1)], sample_size = 400)

# Reference solution
ref = np.load('data/poisson_1.npz')
refsol = jnp.array(ref['usol'])

N_x, N_y = ref['usol'].shape
x, y = ref['x'].flatten(), ref['y'].flatten()
X, Y = jnp.meshgrid(x, y, indexing='ij')
coords = jnp.hstack([X.flatten()[:, None], Y.flatten()[:, None]])

In [ ]:
alpha = 1.0
beta = 1.0

RESULTS[1] = dict()

### Ablation 1: Only RBA (No RAD)

In [ ]:
# PDE Loss
def pde_loss(model, l_E, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Get new RBA weights
    abs_res = jnp.abs(residuals)
    l_E_new = (RBA_gamma*l_E) + (RBA_eta*abs_res/jnp.max(abs_res)) # shape (batch_size, 1)

    # Multiply by RBA weights
    w_resids = l_E_new * residuals # shape (batch_size, 1)

    # Weighted loss
    weighted_loss = jnp.mean(w_resids**2)

    return weighted_loss, l_E_new


@nnx.jit
def train_step(model, optimizer, collocs, l_E):

    # PDE loss
    (loss, l_E_new), grads = nnx.value_and_grad(pde_loss, has_aux=True)(model, l_E, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss, l_E_new

In [ ]:
RESULTS[1]["Ablation 1"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[1]["Ablation 1"][idx] = dict()
    
    # Initialize RBA weights - full pool
    l_E_pool = jnp.ones((collocs_pool.shape[0], 1))

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    l_E = l_E_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss, l_E = train_step(model, optimizer, collocs, l_E)

    tack = time.time()
        
    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[1]["Ablation 1"][idx]['time'] = tack-tick
    RESULTS[1]["Ablation 1"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 1
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

### Ablation 2: No RBA (Only RAD)

In [ ]:
# PDE Loss
def pde_loss(model, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Weighted loss
    loss = jnp.mean(residuals**2)

    return loss


@nnx.jit
def train_step(model, optimizer, collocs):

    # PDE loss
    loss, grads = nnx.value_and_grad(pde_loss, has_aux=False)(model, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss


@nnx.jit
def get_RAD_indices(model, collocs_pool, old_indices):

    # Get full residuals
    resids = pde_res(model, collocs_pool)
    
    # Get absolute
    wa_resids = jnp.abs(resids)

    # Raise to power rad_a
    ea = jnp.power(wa_resids, rad_a)
    
    # Divide by mean and add rad_c
    px = (ea/jnp.mean(ea)) + rad_c
    
    # Normalize
    px_norm = (px / jnp.sum(px))[:,0]

    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=px_norm, seed=seed)

    return sorted_indices

In [ ]:
RESULTS[1]["Ablation 2"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[1]["Ablation 2"][idx] = dict()

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss = train_step(model, optimizer, collocs)
    
        # Perform RAD
        if (epoch != 0) and (epoch % f_resample == 0):

            # Get new indices after resampling
            sorted_indices = get_RAD_indices(model, collocs_pool, sorted_indices)
            # Set new batch of collocs
            collocs = collocs_pool[sorted_indices]

    tack = time.time()

    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[1]["Ablation 2"][idx]['time'] = tack-tick
    RESULTS[1]["Ablation 2"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 2
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

### Ablation 3: No RBA & No RAD (Nothing Adaptive)

In [ ]:
# PDE Loss
def pde_loss(model, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Weighted loss
    loss = jnp.mean(residuals**2)

    return loss


@nnx.jit
def train_step(model, optimizer, collocs):

    # PDE loss
    loss, grads = nnx.value_and_grad(pde_loss, has_aux=False)(model, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss

In [ ]:
RESULTS[1]["Ablation 3"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[1]["Ablation 3"][idx] = dict()

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss = train_step(model, optimizer, collocs)

    tack = time.time()

    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[1]["Ablation 3"][idx]['time'] = tack-tick
    RESULTS[1]["Ablation 3"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 3
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

In [ ]:
with open(result_file, "rb") as f:
    RESULTS = pickle.load(f)

with open(os.path.join(results_dir, "benchmarks_poisson.pkl"), "rb") as f:
    rbn = pickle.load(f)

rbn = rbn[1][(alpha,beta)]

In [ ]:
summarize_results(RESULTS[1], rbn)

## $\omega = 2$

In [ ]:
from src.equations import poisson_2_res as pde_res

In [ ]:
# Collocation points and ICs
collocs_pool = _get_pde_collocs(ranges = [(-1,1), (-1,1)], sample_size = 400)

# Reference solution
ref = np.load('data/poisson_2.npz')
refsol = jnp.array(ref['usol'])

N_x, N_y = ref['usol'].shape
x, y = ref['x'].flatten(), ref['y'].flatten()
X, Y = jnp.meshgrid(x, y, indexing='ij')
coords = jnp.hstack([X.flatten()[:, None], Y.flatten()[:, None]])

In [ ]:
alpha = 1.0
beta = 0.0

RESULTS[2] = dict()

### Ablation 1: Only RBA (No RAD)

In [ ]:
# PDE Loss
def pde_loss(model, l_E, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Get new RBA weights
    abs_res = jnp.abs(residuals)
    l_E_new = (RBA_gamma*l_E) + (RBA_eta*abs_res/jnp.max(abs_res)) # shape (batch_size, 1)

    # Multiply by RBA weights
    w_resids = l_E_new * residuals # shape (batch_size, 1)

    # Weighted loss
    weighted_loss = jnp.mean(w_resids**2)

    return weighted_loss, l_E_new


@nnx.jit
def train_step(model, optimizer, collocs, l_E):

    # PDE loss
    (loss, l_E_new), grads = nnx.value_and_grad(pde_loss, has_aux=True)(model, l_E, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss, l_E_new

In [ ]:
RESULTS[2]["Ablation 1"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[2]["Ablation 1"][idx] = dict()
    
    # Initialize RBA weights - full pool
    l_E_pool = jnp.ones((collocs_pool.shape[0], 1))

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    l_E = l_E_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss, l_E = train_step(model, optimizer, collocs, l_E)

    tack = time.time()
        
    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[2]["Ablation 1"][idx]['time'] = tack-tick
    RESULTS[2]["Ablation 1"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 1
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

### Ablation 2: No RBA (Only RAD)

In [ ]:
# PDE Loss
def pde_loss(model, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Weighted loss
    loss = jnp.mean(residuals**2)

    return loss


@nnx.jit
def train_step(model, optimizer, collocs):

    # PDE loss
    loss, grads = nnx.value_and_grad(pde_loss, has_aux=False)(model, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss


@nnx.jit
def get_RAD_indices(model, collocs_pool, old_indices):

    # Get full residuals
    resids = pde_res(model, collocs_pool)
    
    # Get absolute
    wa_resids = jnp.abs(resids)

    # Raise to power rad_a
    ea = jnp.power(wa_resids, rad_a)
    
    # Divide by mean and add rad_c
    px = (ea/jnp.mean(ea)) + rad_c
    
    # Normalize
    px_norm = (px / jnp.sum(px))[:,0]

    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=px_norm, seed=seed)

    return sorted_indices

In [ ]:
RESULTS[2]["Ablation 2"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[2]["Ablation 2"][idx] = dict()

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss = train_step(model, optimizer, collocs)
    
        # Perform RAD
        if (epoch != 0) and (epoch % f_resample == 0):

            # Get new indices after resampling
            sorted_indices = get_RAD_indices(model, collocs_pool, sorted_indices)
            # Set new batch of collocs
            collocs = collocs_pool[sorted_indices]

    tack = time.time()

    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[2]["Ablation 2"][idx]['time'] = tack-tick
    RESULTS[2]["Ablation 2"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 2
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

### Ablation 3: No RBA & No RAD (Nothing Adaptive)

In [ ]:
# PDE Loss
def pde_loss(model, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Weighted loss
    loss = jnp.mean(residuals**2)

    return loss


@nnx.jit
def train_step(model, optimizer, collocs):

    # PDE loss
    loss, grads = nnx.value_and_grad(pde_loss, has_aux=False)(model, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss

In [ ]:
RESULTS[2]["Ablation 3"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[2]["Ablation 3"][idx] = dict()

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss = train_step(model, optimizer, collocs)

    tack = time.time()

    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[2]["Ablation 3"][idx]['time'] = tack-tick
    RESULTS[2]["Ablation 3"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 3
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

In [ ]:
with open(result_file, "rb") as f:
    RESULTS = pickle.load(f)

with open(os.path.join(results_dir, "benchmarks_poisson.pkl"), "rb") as f:
    rbn = pickle.load(f)

rbn = rbn[2][(alpha,beta)]

In [ ]:
summarize_results(RESULTS[2], rbn)

## $\omega = 4$

In [ ]:
from src.equations import poisson_4_res as pde_res

In [ ]:
# Collocation points and ICs
collocs_pool = _get_pde_collocs(ranges = [(-1,1), (-1,1)], sample_size = 400)

# Reference solution
ref = np.load('data/poisson_4.npz')
refsol = jnp.array(ref['usol'])

N_x, N_y = ref['usol'].shape
x, y = ref['x'].flatten(), ref['y'].flatten()
X, Y = jnp.meshgrid(x, y, indexing='ij')
coords = jnp.hstack([X.flatten()[:, None], Y.flatten()[:, None]])

In [ ]:
alpha = 0.0
beta = 1.0

RESULTS[4] = dict()

### Ablation 1: Only RBA (No RAD)

In [ ]:
# PDE Loss
def pde_loss(model, l_E, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Get new RBA weights
    abs_res = jnp.abs(residuals)
    l_E_new = (RBA_gamma*l_E) + (RBA_eta*abs_res/jnp.max(abs_res)) # shape (batch_size, 1)

    # Multiply by RBA weights
    w_resids = l_E_new * residuals # shape (batch_size, 1)

    # Weighted loss
    weighted_loss = jnp.mean(w_resids**2)

    return weighted_loss, l_E_new


@nnx.jit
def train_step(model, optimizer, collocs, l_E):

    # PDE loss
    (loss, l_E_new), grads = nnx.value_and_grad(pde_loss, has_aux=True)(model, l_E, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss, l_E_new

In [ ]:
RESULTS[4]["Ablation 1"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[4]["Ablation 1"][idx] = dict()
    
    # Initialize RBA weights - full pool
    l_E_pool = jnp.ones((collocs_pool.shape[0], 1))

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    l_E = l_E_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss, l_E = train_step(model, optimizer, collocs, l_E)

    tack = time.time()
        
    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[4]["Ablation 1"][idx]['time'] = tack-tick
    RESULTS[4]["Ablation 1"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 1
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

### Ablation 2: No RBA (Only RAD)

In [ ]:
# PDE Loss
def pde_loss(model, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Weighted loss
    loss = jnp.mean(residuals**2)

    return loss


@nnx.jit
def train_step(model, optimizer, collocs):

    # PDE loss
    loss, grads = nnx.value_and_grad(pde_loss, has_aux=False)(model, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss


@nnx.jit
def get_RAD_indices(model, collocs_pool, old_indices):

    # Get full residuals
    resids = pde_res(model, collocs_pool)
    
    # Get absolute
    wa_resids = jnp.abs(resids)

    # Raise to power rad_a
    ea = jnp.power(wa_resids, rad_a)
    
    # Divide by mean and add rad_c
    px = (ea/jnp.mean(ea)) + rad_c
    
    # Normalize
    px_norm = (px / jnp.sum(px))[:,0]

    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=px_norm, seed=seed)

    return sorted_indices

In [ ]:
RESULTS[4]["Ablation 2"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[4]["Ablation 2"][idx] = dict()

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss = train_step(model, optimizer, collocs)
    
        # Perform RAD
        if (epoch != 0) and (epoch % f_resample == 0):

            # Get new indices after resampling
            sorted_indices = get_RAD_indices(model, collocs_pool, sorted_indices)
            # Set new batch of collocs
            collocs = collocs_pool[sorted_indices]

    tack = time.time()

    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[4]["Ablation 2"][idx]['time'] = tack-tick
    RESULTS[4]["Ablation 2"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 2
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

### Ablation 3: No RBA & No RAD (Nothing Adaptive)

In [ ]:
# PDE Loss
def pde_loss(model, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Weighted loss
    loss = jnp.mean(residuals**2)

    return loss


@nnx.jit
def train_step(model, optimizer, collocs):

    # PDE loss
    loss, grads = nnx.value_and_grad(pde_loss, has_aux=False)(model, collocs)

    # Optimizer step
    optimizer.update(grads)

    return loss

In [ ]:
RESULTS[4]["Ablation 3"] = dict()

for idx, run in enumerate([0, 7, 42]):

    RESULTS[4]["Ablation 3"][idx] = dict()

    # Get starting collocation points & RBA weights
    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
    
    collocs = collocs_pool[sorted_indices]
    
    # Get opt_type
    opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

    # Define model
    model = PoissonModel(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_blocks = num_blocks, D = D,
                       init_scheme = init_scheme, alpha = alpha, beta = beta, ref = None,
                       period_axes = period_axes, rff_std = None, sine_D = sine_D, seed = seed+run)

    if idx == 0:
        print(f"Initialized model with {count_params(model)} parameters.")

    # Set optimizer
    optimizer = nnx.Optimizer(model, opt_type)
    
    tick = time.time()

    # Start training
    for epoch in range(num_epochs):
    
        loss = train_step(model, optimizer, collocs)

    tack = time.time()

    l2error = model_eval(model, coords, refsol)

    print(f"\tRun = {idx}\t L^2 = {l2error:.2e}\t Loss = {loss:.2e}\t Time = {(tack-tick)/60:.2f} mins")

    RESULTS[4]["Ablation 3"][idx]['time'] = tack-tick
    RESULTS[4]["Ablation 3"][idx]['l2'] = l2error.item()

In [ ]:
# Checkpoint 3
with open(result_file, "wb") as f:
    pickle.dump(RESULTS, f)

In [ ]:
with open(result_file, "rb") as f:
    RESULTS = pickle.load(f)

with open(os.path.join(results_dir, "benchmarks_poisson.pkl"), "rb") as f:
    rbn = pickle.load(f)

rbn = rbn[4][(alpha,beta)]

In [ ]:
summarize_results(RESULTS[4], rbn)